In [1]:
import numpy as np
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.geometry import MeshcatVisualizer, StartMeshcat
from pydrake.multibody.parsing import Parser
from pydrake.math import RigidTransform, RotationMatrix
from pydrake.visualization import ApplyVisualizationConfig, VisualizationConfig, AddFrameTriadIllustration

# from manipulation.scenarios import SetColor

In [2]:
# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7001


In [3]:
builder = DiagramBuilder()

plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)
parser = Parser(plant, scene_graph)
parser.SetAutoRenaming(True)

# Add the models to the plant.

# Transparent objects
# brick = parser.AddModelFromFile("models/cube_transparent.sdf")
# grasp1 = parser.AddModelFromFile("models/wsg_transparent.sdf")
# grasp2 = parser.AddModelFromFile("models/wsg_transparent.sdf")
# grasp3 = parser.AddModelFromFile("models/wsg_transparent.sdf")
# grasp4 = parser.AddModelFromFile("models/wsg_transparent.sdf")

grasp1 = parser.AddModels(file_name="my_sdfs/wsg.sdf")[0]
grasp2 = parser.AddModels(file_name="my_sdfs/wsg.sdf")[0]
grasp3 = parser.AddModels(file_name="my_sdfs/wsg.sdf")[0]
grasp4 = parser.AddModels(file_name="my_sdfs/wsg.sdf")[0]

brick = parser.AddModels(file_name="my_sdfs/cube.sdf")[0]
plant.Finalize()

bodies_list1 = [grasp1, grasp2, grasp3, grasp4]

In [4]:
B_O = plant.GetBodyByName("base_link", brick)
B_Ggrasp1 = plant.GetBodyByName("body", grasp1)
B_Ggrasp2 = plant.GetBodyByName("body", grasp2)
B_Ggrasp3 = plant.GetBodyByName("body", grasp3)
B_Ggrasp4 = plant.GetBodyByName("body", grasp4)

bodies_list = [B_O, B_Ggrasp1, B_Ggrasp2, B_Ggrasp3, B_Ggrasp4]

In [5]:

for body in bodies_list:
    if body == B_O:
        AddFrameTriadIllustration(
            plant=plant,
            scene_graph=scene_graph,
            body=body,
            length=0.09
        )
        continue
    AddFrameTriadIllustration(
        plant=plant,
        scene_graph=scene_graph,
        body=body,
        length=0.09,
        # opacity=0.2,
    )

In [6]:
meshcat.Delete()
meshcat.SetProperty("/Background", "visible", False)
MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

In [7]:
diagram = builder.Build()
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(context)

In [8]:

# Get the current object, O, pose
world_body = plant.world_body()
X_WO = plant.EvalBodyPoseInWorld(plant_context, world_body)

rotation_constraints = np.linspace(0, np.pi, len(bodies_list)-1)

p_GgraspO = [0, .11, 0]
R_GgraspO = {}
X_GgraspO = {}
X_OGgrasp = {}
X_WGgrasp = {}

for i in range(len(rotation_constraints)):
    R_GgraspO[i] = RotationMatrix.MakeXRotation(rotation_constraints[i]).multiply(RotationMatrix.MakeZRotation(np.pi / 2))
    X_GgraspO[i] = RigidTransform(R_GgraspO[i], p_GgraspO)
    X_OGgrasp[i] = X_GgraspO[i].inverse()
    X_WGgrasp[i] = X_WO.multiply(X_OGgrasp[i])


for i in range(len(rotation_constraints)):
    plant.SetFreeBodyPose(plant_context, bodies_list[i+1], X_WGgrasp[i])
    plant.GetJointByName("left_finger_sliding_joint", bodies_list1[i]).set_translation(
        plant_context, -0.025
    )
    plant.GetJointByName("right_finger_sliding_joint", bodies_list1[i]).set_translation(
        plant_context, 0.025
    )


In [9]:
diagram.ForcedPublish(context)